<a href="https://colab.research.google.com/github/hajaulee/correcting-rotation-of-documents/blob/master/correcting_rotation_of_documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

In [0]:
!ls -ial '/content/drive/My Drive/correcting-rotation-of-documents'

In [13]:
import cv2 as cv
import glob
import os
import time
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.client import device_lib
from keras.layers import Convolution2D, Dropout, Dense, Flatten, MaxPooling2D, Input, BatchNormalization
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras import metrics
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [0]:
working_path = '/content/drive/My Drive/correcting-rotation-of-documents'
image_shape = (200,200,1)
nb_classes = 4

In [0]:
def indices_to_one_hot(data, nb_classes):
    """Convert an iterable of indices to one-hot encoded labels."""
    targets = np.array(data).reshape(-1)
    return np.eye(nb_classes)[targets]

In [0]:
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

In [0]:
def load_data(folder):
    X, labels, names = [],[], []
    list_images_file = glob.glob(os.path.join(folder, "*.jpg"))
    for filename in list_images_file:
        img = cv.imread(filename, cv.IMREAD_GRAYSCALE)
        img = cv.resize(img, image_shape[:2])
        img = np.asarray(img)
        img = img.reshape(image_shape)
        label = os.path.basename(filename)[0]
        X.append(img)
        labels.append(label)
        names.append(os.path.basename(filename))
        print ('Loaded', filename)
    return np.asarray(X), np.asarray(labels), names

In [0]:
def get_model(nb_classes):
    # size of pooling area for max pooling
    pool_size = (2, 2)
    # convolution kernel size
    kernel_size = (3,3)
    # convolution strides
    strides = (1,1)

    # model definition
    input = Input(shape=image_shape)
    x = Convolution2D(96, kernel_size,strides=strides,
                      activation='relu')(input)
    x = MaxPooling2D(pool_size=pool_size)(x)
    x = BatchNormalization()(x)
    x = Convolution2D(64, kernel_size, strides=strides,
                      activation='relu')(x)
    x = MaxPooling2D(pool_size=pool_size)(x)
    x = BatchNormalization()(x)
    x = Convolution2D(32, kernel_size, strides=strides,
                      activation='relu')(x)
    x = MaxPooling2D(pool_size=pool_size)(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.33)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(rate=0.25)(x)
    x = Dense(nb_classes, activation='softmax')(x)

    model = Model(inputs=[input], outputs=x)

    model.summary()
    # model compilation
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    return model

In [0]:
def train(model, train_, test_):
    # training parameters
    batch_size = 128
    nb_epoch = 50

    output_filename = 'model.json'
    # callbacks
    checkpointer = ModelCheckpoint(
        filepath=output_filename,
        save_best_only=True
    )
    early_stopping = EarlyStopping(patience=2)
    tensorboard = TensorBoard()


    # training loop
    with tf.device('/gpu:0'):
        histories = model.fit(train_[0], train_[1],
            epochs=nb_epoch,
            batch_size=125,
            validation_data=test_,
            verbose=True,
            shuffle=True,
            callbacks=[checkpointer, early_stopping, tensorboard]
        )

    return model, histories

In [23]:
start_time = time.time()
# Build model
model = get_model(nb_classes)
build_model_time = time.time() - start_time
    

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 200, 200, 1)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 198, 198, 96)      960       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 99, 99, 96)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 99, 99, 96)        384       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 97, 97, 64)        55360     
_________________________________________________________________
max_pooling2d_2 (MaxP

In [29]:
start_time = time.time()
images_path = os.path.join(working_path,'train_dataset')
print('Load images from', images_path)
X_data, Y_data, N_data = load_data(images_path)
X_train, X_test, Y_train, Y_test = train_test_split(X_data, Y_data, test_size=.33, random_state=42)

load_data_time = time.time() - start_time
print('Training with', len(X_train), 'images and validate with', len(X_test), 'images') 

Load images from /content/drive/My Drive/correcting-rotation-of-documents/train_dataset
Loaded /content/drive/My Drive/correcting-rotation-of-documents/train_dataset/270-X51005268262.jpg
Loaded /content/drive/My Drive/correcting-rotation-of-documents/train_dataset/270-X51005268200.jpg
Loaded /content/drive/My Drive/correcting-rotation-of-documents/train_dataset/270-X51005301661.jpg
Loaded /content/drive/My Drive/correcting-rotation-of-documents/train_dataset/270-X51005301659.jpg
Loaded /content/drive/My Drive/correcting-rotation-of-documents/train_dataset/270-X51006414638.jpg
Loaded /content/drive/My Drive/correcting-rotation-of-documents/train_dataset/270-X51005433492.jpg
Loaded /content/drive/My Drive/correcting-rotation-of-documents/train_dataset/270-X51005361900.jpg
Loaded /content/drive/My Drive/correcting-rotation-of-documents/train_dataset/270-X51005361898.jpg
Loaded /content/drive/My Drive/correcting-rotation-of-documents/train_dataset/270-X51005361950.jpg
Loaded /content/drive